# ESPnet 을 이용한 음성인식 실습

ESPnet 툴킷을 이용하여 음성인식을 수행합니다. 본 실습에서는 음성인식 모델의 훈련은 포함하지 않습니다.
이미 훈련된 모델을 이용하여 평가데이터에 대하여 인식을 수행하고 인식률을 측정하는 방법을 설명합니다.

## 환경 설정
먼저 실습환경을 설정합니다. 아래 스크립트는 jupyter 환경을 기준으로 설명합니다.
일반 리눅스 환경에서는 먼저 https://github.com/pkyoung/a1003.git 를 내려받고
jupyter notebook을 실행하여 이 파일을 jupyter환경에서 열어봅니다.
```
git clone https://github.com/pkyoung/a1003.git ~/a1003
conda create -n a1003 pythoon=3.10
conda activate a1003
pip install jupyter
jupyter notebook --notebookdir ~/a1003/local
```

jupyter 환경이 아닌 일반 shell 환경에서도 실행가능합니다. 이 경우
conda 환경 안에서 `!` 로 시작하는 명령어들을 순차적으로 실행합니다.
`!`는 입력하지 않습니다.

1. 홈디렉토리에 a1003 디렉토리가 없으면, https://github.com/pkyoung/a1003.git 의 코드를 홈 디렉토리에 내려받습니다. Colab 환경에서는 홈디렉토리는 /content 입니다. 그 밖의 linux 환경에서는 홈디렉토리는 ~/ 입니다.
2. 실습은 a1003/local 디렉토리에서 수행합니다. 이 디렉토리로 이동합니다.

In [ ]:
# Run this cell to automatically clone the repo
import sys
import os
!pip install pickleshare
if 'google.colab' in sys.modules:
    if not os.path.exists("/content/a1003"):
        !git clone "https://github.com/pkyoung/a1003.git" /content/a1003
    %cd /content/a1003/local
else:
    if not os.path.exists(os.path.expanduser("~/a1003")):
        !git clone "https://github.com/pkyoung/a1003.git" ~/a1003
    %cd ~/a1003/local

## ESPnet 설치

In [ ]:
!pip install torchaudio espnet

Colab환경에서는 numpy를 다른 버전으로 재설치해야합니다. 재설치 후 세션을 재시작하므로 브라우저에서 오류메시지가 나타날 수 있습니다.


In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install numpy==1.26.4
    import os
    os.kill(os.getpid(), 9)

In [ ]:
import sys
if 'google.colab' in sys.modules:
  %cd /content/a1003/local

huggingface에서 테스트용 데이터셋을 내려받습니다.
9명의 화자가 각각 20개의 짧은 문장을 발성한 wav 파일과 정답전사문 파일입니다.

In [ ]:
import os
if not os.path.exists("data/a1003"):
  !git clone https://hf.co/datasets/pkyoung/a1003.git ./data/a1003

내려받은 파일을 확인해봅니다. 디렉토리의 구조는 다음과 같습니다.
```
data/a1003
    ├── README.md
    ├── result_whisper_turbo.txt
    ├── text
    ├── wav.scp
    ├── wav
    │   ├── spk1
    │   │   ├── 1.wav
    │   │   ├── 10.wav
    │   │   ├── 11.wav
    ...
```
`text` 파일과 `wav.scp` 파일은 앞서 설명한 `kaldi` 파일과 같은 형식을 갖습니다. 파일은 다음과 같은 방법으로 들어볼 수 있습니다.

In [ ]:
from IPython.display import Audio
filename = "data/a1003/wav/spk1/1.wav"
Audio(filename, autoplay=False)

## ESPnet을 이용한 음성인식

먼저 모델을 내려받습니다. 아래 모델은 약 8,000시간의 한국어 음성을 이용하여 ESPnet으로 훈련된 attenion/CTC 하이브리드 모델입니다. ma16k2401a 외에 ma16k2401b 모델과 ma16k2401c 모델도 내려받을 수 있습니다. 세 모델이 디코더는 동일한 구조인데, 인코더만 각각 트랜스포머, 컨포머, 이브랜치포머 입니다.

In [ ]:
import os
if not os.path.exists("./model/ma16k2401a"):
  !git clone https://hf.co/pkyoung/ma16k2401a ./models/ma16k2401a

내려받은 모델의 구조를 확인해봅니다.

In [ ]:
!ls -lh models/ma16k2401a/

`espnet-infer.sh` 파일을 열어서 필요한 설정을 변경하고 저장한 후 실행합니다.

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# 수정할 파일 이름
file_name = "espnet-infer.sh"

# 파일 읽기
def read_file(filename):
    with open(filename, "r") as f:
        return f.read()

# 파일 저장
def save_file(change):
    with open(file_name, "w") as f:
        f.write(text_area.value)
    print(f"✅ '{file_name}' 저장 완료!")

# 레이블 (파일 이름 또는 설명)
label = widgets.HTML(value=f"<b>{file_name}</b> 파일 내용:")

# 텍스트 입력창
text_area = widgets.Textarea(
    value=read_file(file_name),
    layout=widgets.Layout(width="100%", height="300px")
)

# 저장 버튼
save_button = widgets.Button(
    description="💾 저장하기",
    button_style="success"
)

save_button.on_click(save_file)

# 화면에 표시
display(label, text_area, save_button)

In [ ]:
!bash espnet-infer.sh

`$odir`에 저장된 결과 파일을 살펴봅니다. 예를 들어 `odir=results`라고 설정한 경우, `results/1best_recog/text` 파일에 인식결과가 기록됩니다.

## ESPnet 모델의 음성인식 오류율 측정
발화별 CER과 평균 CER을 계산합니다.

`data/a1003/result_espnet_ma16k2401a.txt` 파일에 `ma16k2401a` 모델로 인식을 수행한 결과를 저장해 두었습니다. 인식을 모두 수행하지 않은 경우라면 이 파일을 사용해서 오류율을 측정해봅니다.

In [ ]:
if (
    os.path.exists('results/1best_recog/text') and
    sum(1 for _ in open('results/1best_recog/text')) == sum(1 for _ in open('data/a1003/wav.scp'))
):
    !cp results/1best_recog/text ./result.txt
else:
    !cp data/a1003/result_espnet_ma16k2401a.txt ./result.txt


In [ ]:
!python uttcer.py data/a1003/text result.txt

화자별 음성인식 성능을 계산합니다.

In [ ]:
%%bash
for s in spk{1..9}; do
  echo -n "$s " >&2
  python uttcer.py data/a1003/text <(grep "$s/" result.txt) > /dev/null
done

평균 오류율이 높은 화자에 대해서는 몇몇 파일을 들어봅니다. 주석처리된 부분을 바꿔가면서 들어봅니다.

In [ ]:
from IPython.display import Audio
filename = "data/a1003/wav/spk2/utt1.wav"
#filename = "data/a1003/wav/spk7/recording_00.wav"
#filename = "data/a1003/wav/spk3/data15.wav"

Audio(filename, autoplay=False)

문장부호를 제외한 CER을 계산합니다.

In [ ]:
!python uttwer.py <(python pp.py -s data/a1003/text) <(python pp.py -s result.txt) > /dev/null

## ESPnet 모델별 음성인식 성능 및 속도

각 모델별 모델크기, 인식 소요 시간 및 인식 성능은 다음 표와 같습니다.
인식소요시간 및 인식성능은 환경에 따라 달라질 수 있습니다. (WER1, CER1: 문장부호 제외 전/WER2, CER2: 문장부호 제외 후)

| Model          | Size | Elapsed | RTF | WER1  |  CER1 | WER2  | CER2 |
| ---            | ---  | ---     | --- | ---   |  ---  |  ---  | ---  |
| ma16k2401a     | 280M | 4m8s    |0.15 | 28.20 | 10.09 | 20.10 | 7.36 |
| ma16k2401b     | 425M | 4m5s    |0.15 | 28.37 |  9.79 | 19.55 | 6.95 |
| ma16k2401c     | 550M | 4m21s   |0.16 | 26.44 |  8.89 | 17.45 | 5.88 |

